In [1]:
from src.stepper import Stepper
import logging

# Configure debug logging
logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


with Stepper("COM5") as motor:
    motor.enable()

    # print(f"Version: {motor.get_version()}")
    # print(f"Position: {motor.get_position()}")
    # print(f"Position error: {motor.get_position_error()}")

    # print("Drive config:")
    # for items in motor.get_drive_config().items():
    #     print(items)

    # print(f"Status: {motor.get_status()}")

    print("System status:")
    for items in motor.get_system_status().items():
        print(items)

    print(f"Homing status: {motor.get_homing_status()}")
    print(f"Motor status: {motor.get_encoder_calibration_status()}")

    print(f"Bus voltage: {motor.get_bus_voltage()}")
    print(f"Phase current: {motor.get_phase_current()}")


2024-11-27 10:20:58,477 - StepperMotor - DEBUG - Command: Enable motor (store=0, sync=0)
TX: 01 F3 AB 01 00 6B
2024-11-27 10:20:58,481 - StepperMotor - DEBUG - Response to Enable motor (store=0, sync=0):
RX: 01 F3 02 6B
2024-11-27 10:20:58,481 - StepperMotor - DEBUG - Command: Read system status
TX: 01 43 7A 6B
2024-11-27 10:20:58,487 - StepperMotor - DEBUG - Response to Read system status:
RX: 01 43 1F 09 2E AC 00 49 2A 9C 01 00 00 7F F6 00 00 00 01 00 00 80 0E 00 00 00 00 18 07 03 6B
2024-11-27 10:20:58,488 - StepperMotor - DEBUG - System status raw data: 09 2E AC 00 49 2A 9C 01 00 00 7F F6 00 00 00 01 00 00 80 0E 00 00 00 00 18 07 03
2024-11-27 10:20:58,488 - StepperMotor - DEBUG - Command: Read homing status
TX: 01 3B 6B
2024-11-27 10:20:58,492 - StepperMotor - DEBUG - Response to Read homing status:
RX: 01 3B 07 6B


System status:
('bus_voltage', 2.35)
('phase_current', 44.032)
('encoder_value', 18730)
('target_position', Position(steps=-16777343, revolutions=-257, degrees=0.6976318359375))
('realtime_speed', 0)
('realtime_position', Position(steps=16777344, revolutions=256, degrees=0.703125))
('position_error', {'steps': 0, 'degrees': 0.0})
('ready_status', {'encoder_ready': False, 'calibration_ready': False, 'homing_in_progress': False, 'homing_failed': True})
('motor_status', {'enabled': True, 'position_reached': True, 'stalled': True, 'protection_triggered': False})


TypeError: 'NoneType' object is not subscriptable

In [8]:
sum((1, 1, 2, 4, 2, 2, 2, 1))

15

In [2]:
import time

for i in range(10):
    with StepperMotor("COM5") as motor:
        motor.enable()
        motor.set_position(steps=80, speed=3000, acceleration=255)
    time.sleep(0.5)


2024-11-27 09:24:27,358 - StepperMotor - DEBUG - Command: Enable motor (store=0, sync=0)
TX: 01 F3 AB 01 00 6B
2024-11-27 09:24:27,362 - StepperMotor - DEBUG - Response to Enable motor (store=0, sync=0):
RX: 01 F3 02 6B
2024-11-27 09:24:27,363 - StepperMotor - DEBUG - Command: Move relative 80 steps at 3000 RPM (dir=1, accel=255)
TX: 01 FD 01 0B B8 FF 00 00 00 50 00 00 6B
2024-11-27 09:24:27,363 - StepperMotor - DEBUG - Response to Move relative 80 steps at 3000 RPM (dir=1, accel=255):
RX: 01 FD E2 6B


StatusError: Motor stalled or disabled

In [4]:
with StepperMotor("COM5") as motor:
    try:
        motor.enable()
        logger.debug("Motor enabled successfully")

    except Exception as e:
        logger.error(f"Communication error: {str(e)}", exc_info=True)
        status = motor.get_status()

    try:
        motor.home(mode=0)  # Home to nearest position
    except Exception as e:
        logger.error(f"Homing failed: {str(e)}")
        raise
    motor.set_zero()  # Set current position as zero

    # Move with acceleration
    motor.set_position(
        steps=32000,  # 10 revolutions
        speed=1500,  # 1500 RPM
        acceleration=10,  # Smooth acceleration
    )

    # Monitor status
    status = motor.get_status()
    if status.stalled:
        motor.clear_stall()
        motor.emergency_stop()


2024-11-27 08:49:42,662 - StepperMotor - DEBUG - Command: Enable motor (store=0, sync=0)
TX: 01 F3 AB 01 00 6B
2024-11-27 08:49:42,666 - StepperMotor - DEBUG - Response to Enable motor (store=0, sync=0):
RX: 01 F3 02 6B
2024-11-27 08:49:42,666 - __main__ - DEBUG - Motor enabled successfully
2024-11-27 08:49:42,667 - StepperMotor - DEBUG - Command: Read motor status
TX: 01 3A 6B
2024-11-27 08:49:42,669 - StepperMotor - DEBUG - Response to Read motor status:
RX: 01 3A 03 6B
2024-11-27 08:49:42,670 - StepperMotor - DEBUG - Command: Home motor (single-turn nearest)
TX: 01 9A 00 00 6B
2024-11-27 08:49:42,674 - StepperMotor - DEBUG - Response to Home motor (single-turn nearest):
RX: 01 9A 02 6B
2024-11-27 08:49:42,675 - StepperMotor - DEBUG - Command: Set zero position (store=1)
TX: 01 93 88 01 6B


In [43]:
def define_config_type(minimum: int, maximum: int, default: int, digits: int):
    class RangedInt(int):
        def __new__(cls, value: int = default, *args, **kwargs):
            instance = int.__new__(cls, value, *args, **kwargs)
            cls.digits = digits
            if not minimum <= instance <= maximum:
                raise ValueError("Value out of range")
            return instance

        @property
        def bytes(self) -> bytes:
            return self.to_bytes(self.digits, "big")

    return RangedInt


In [10]:
from dataclasses import dataclass, field

@dataclass
class MetaParam:
    minimum: int
    maximum: int
    default: int
    digits: int

@dataclass
class Named:
    name: str
    code: int = field(default=0, init=False)


@dataclass
class NamedMetaParam(Named, MetaParam):
    pass


a = NamedMetaParam(name="a", minimum=0, maximum=100, default=50, digits=2)

a.code


0

In [9]:
from dataclasses import dataclass


class RangedInt(int):
    MetaParam: Named

    def __new__(cls, value: int) -> "RangedInt":
        if not hasattr(cls, "minimum") or not hasattr(cls, "maximum"):
            raise TypeError("RangedInt cannot be instantiated directly")
        if not cls.minimum <= value <= cls.maximum:
            raise ValueError(f"Value must be between {cls.minimum} and {cls.maximum}")
        return super().__new__(cls, value)


def create_ranged_int(name: str, minimum: int, maximum: int) -> type[RangedInt]:
    """Creates a new RangedInt subclass with specified bounds.

    :param name: Name of the new class
    :param minimum: Minimum allowed value
    :param maximum: Maximum allowed value
    :raises ValueError: If minimum > maximum
    :return: New RangedInt subclass
    """
    if minimum > maximum:
        raise ValueError("Minimum value cannot be greater than maximum")

    return type(
        name,
        (RangedInt,),
        {
            "minimum": minimum,
            "maximum": maximum,
            "__doc__": f"Integer constrained between {minimum} and {maximum}",
        },
    )


# Usage example:
Speed = create_ranged_int("Speed", 0, 255)
Acceleration = create_ranged_int("Acceleration", 0, 1288)

# Now you can use them like this:
speed = Speed(100)  # Valid
accel = Acceleration(1000)  # Valid

<class '__main__.RangedInt'>
